# Westeros Tutorial
## Adding representation of renewables - Part 2: - Introducing `flexible generation`

This tutorial, which demonstrates how to apply various model features to provide a more realistic representation of renewable energy integration in the energy system, is comprised of three parts. In the first part we introduced `firm capacity` constraints to ensure that conventional electricity generation plants supplied sufficient backup capacity to allow for high renewable electricity generation penetration. In this tutorial we will address `flexible generation` i.e., a powerplants capacility to ramp up generation in order to respond to load fluctuations.

Note, that for this turoial, we will be starting off from the scenario we developed as part of the '*Westeros Tutorial III*', which use an `emission_price` as opposed to a `bound_emission`.  This is because using bound on emission renders an infeasibility.

Further information can be found in https://doi.org/10.1016/j.esr.2013.01.001 (*Sullivan et al., 2013*)

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

## Load the existing scenario '*carbon_tax*' and clone to a new scenario '*flexible_generation*' to which we will apply the `flexible generation` constraint

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='carbon_tax')
scen = base.clone(model, 'flexible_generation',
                  'illustration of flexible-generation formulation', keep_solution=False)
scen.check_out()

## Retrieve parameters to perform subsequent addition of parameters

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

# Describing the Renewable Technologies  Flexibility

Electric-sector flexibility is represented as follows: each generating technology is assigned a coefficient between −1 and 1 representing (if positive) the fraction of generation from that technology that is considered to be flexible or (if negative) the additional flexible generation required for each unit of generation from that technology. A negative coefficient on load reveals a parameterization of the amount of flexible energy the system requires solely to meet changes and uncertainty in load. (*Sullivan et al., 2013*)

The coefficients used are derived in *Sullivan et al., 2013*.

| Technology | Flexibility parameter |
| ---------- | --------------------- |
| Load | −0.1 |
| Wind | −0.08 |
| Solar PV | −0.05 |
| Geothermal | 0 |
| Nuclear | 0 |
| Coal | 0.15 |
| Biopower | 0.3 |
| Gas-CC | 0.5 |
| Hydropower | 0.5 |
| H2 Electrolysis | 0.5 |
| Oil/gas steam | 1 |
| Gas-CT | 1 |
| Electricity storage | 1 |

Based on the above listed coefficients, our `wind_ppl` will have a load demand of 8% of its activity. Likewise the elctricity grid has a load demand of 10%, while on the `coal_ppl` can provide 15% of its activity as flexibility.  

Recall that in the previous tutorial two rating bins were introduced for `wind_ppl`, depicting the different firm capacity requirements as market penetration increases.  We will again use the two rating bins previously defined. Here we make the assumption that the above flexibility parameter (8%) applies to the larger of two rating bins (`r2`), and assume that the smaller rating bin (`r1`) has a lower flexibility demand of 6%, correlating to the fact that the first 20% of installed capacity required contributes more to firm capacity.  

In [ ]:
base_flexibility_factor = pd.DataFrame({
        'node_loc': country,
        'commodity': 'electricity',
        'level' : 'secondary',
        'mode': 'standard',
        'unit': '???',
        'time': 'year',
        'year_vtg': vintage_years,
        'year_act': act_years,
})

base_rating = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '???',
        'time': 'year',
        'year_act': model_horizon})

In [ ]:
# add the ratings as a set 
scen.add_set('rating', ['r1', 'r2'])

# For the Load 
flexibility_factor = make_df(base_flexibility_factor, technology= 'grid', rating= 'unrated', value = -0.1)
scen.add_par('flexibility_factor',flexibility_factor)

# For the Wind PPL
rating_bin = make_df(base_rating, technology= 'wind_ppl', value = 0.2, rating= 'r1')
scen.add_par('rating_bin', rating_bin)

flexibility_factor = make_df(base_flexibility_factor, technology= 'wind_ppl', rating= 'r1', value = -0.06)
scen.add_par('flexibility_factor',flexibility_factor)

rating_bin = make_df(base_rating, technology= 'wind_ppl', value = 0.8, rating= 'r2')
scen.add_par('rating_bin', rating_bin)

flexibility_factor = make_df(base_flexibility_factor, technology= 'wind_ppl', rating= 'r2', value = -0.08)
scen.add_par('flexibility_factor',flexibility_factor)

# For the Coal PPL
flexibility_factor = make_df(base_flexibility_factor, technology= 'coal_ppl', rating= 'unrated', value = 1)
scen.add_par('flexibility_factor',flexibility_factor)

### commit and solve

In [ ]:
scen.commit(comment='define parameters for flexibile-generation implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700) # scenario: 'flexible generation' (carbon_tax scenario with flexible generation constraint)
b = Plots(base, country, firstyear=700) # scenario: 'carbon_tax' (without flexible generation)

## Activity
***
The impact of adding the flexiblity constraint to electricity generation technologies on the model behavior can be observed by comparing the activity levels between the two scenarios. In the '*carbon_tax*' scenario, electricity in 720 is generated solely by the `wind_ppl`.  When adding the flexibility constraint, the `coal_ppl` is also required to generate electricity in order to provide activity to meet the demanded flexibility by the `load` and the `wind_ppl`.  In the previous timesteps, the `coal_ppl` activity is already sufficient to meet the demanded flexibility, so there is no change in activity.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*flexible_generation*'

In [ ]:
rep.set_filters(t=["coal_ppl", "wind_ppl"])
rep.get("plot activity")

## Capacity
***
The additional activity in 720 by the coal_ppl in the '*flexible_generation*' scenario implies that additional capacity is required in that year, increasing the price of electricity in 720 as can be seen from the last two plots comparing the prices of light.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*flexible_generation*'

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Prices
***

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_prices(subset=['light'], baseyear=True)

### Scenario: '*flexible_generation*'

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()